In [8]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM 
from torch import nn
import pandas as pd
import pyarrow as pa
from torch.optim import Adam
import numpy as np

In [9]:
df = pd.read_parquet("small_tokenized_data.parquet")
IDS = df["input_ids"].tolist()
newIDS=[]
for each in IDS:
    #end_index = next((i for i, x in enumerate(each) if x==0), len(each))
    # Remove the trailing 0's
    #each = each[:end_index]
    # Convert the input to a tensor
    input_tensor = torch.tensor(each).unsqueeze(0)
    newIDS.append(each)

In [10]:
#get the star count AKA the y from half_data
df = pd.read_parquet("small_tokenized_data.parquet")
IDS = df["stars"].tolist()
starIDS=[]
for each in IDS:
    #end_index = next((i for i, x in enumerate(each) if x==0), len(each))
    # Remove the trailing 0's
    #each = each[:end_index]
    # Convert the input to a tensor
    input_tensor = torch.tensor(each).unsqueeze(0)
    starIDS.append(each)

In [11]:
bert = BertModel.from_pretrained('bert-base-uncased')

In [12]:
#load data in batches for bert
bertData = torch.utils.data.TensorDataset(torch.tensor(newIDS), torch.tensor(starIDS))

# set up a data loader to get batches from our dataset
bertLoader = torch.utils.data.DataLoader(bertData, batch_size = 16, shuffle = True)

In [19]:
#PyTorch tensor with first 3 reviews from our training set and pass it to it
#this is the BERTs final layer output for each token
i = 0
list_pool = []
for batch in bertLoader:
    if i < 3:
        i = i+1
        print(i)
        y, pooled = bert(batch[0], output_all_encoded_layers=False)
        list_pool.append(pooled)
        
    else:
        break


# NEXT THING TO DO
#neural net model to go from pooled (vetors) to labels

1
2
3


In [20]:
print(pooled.shape)
print(y.shape)

torch.Size([16, 768])
torch.Size([16, 512, 768])


In [21]:
print(pooled)

tensor([[-0.4651, -0.6454, -0.9978,  ..., -0.9722, -0.4942,  0.2123],
        [-0.4316, -0.5366, -0.9880,  ..., -0.9341, -0.4829,  0.3122],
        [-0.4558, -0.7208, -0.9996,  ..., -0.9930, -0.5149,  0.1792],
        ...,
        [-0.5408, -0.2726, -0.6107,  ..., -0.7237, -0.5368,  0.7631],
        [-0.6096, -0.6561, -0.9992,  ..., -0.9938, -0.5293,  0.4072],
        [-0.3362, -0.4695, -0.9950,  ..., -0.9652, -0.4043,  0.0036]],
       grad_fn=<TanhBackward0>)


In [23]:
import torch
from sklearn.model_selection import train_test_split
# convert our data into a torch dataset
#need y data from the file
ratings = torch.tensor(starIDS).float()
#x_train, x_test, y_train, y_test = train_test_split(pooled.detach(), ratings, test_size=0.15)

x_train, x_test, y_train, y_test = train_test_split(list_pool, ratings, test_size=0.15)
data = torch.utils.data.TensorDataset(x_train, y_train)

# set up a data loader to get batches from our dataset
loader = torch.utils.data.DataLoader(data, batch_size = 16, shuffle = True)

ValueError: Found input variables with inconsistent numbers of samples: [3, 10000]

In [ ]:
model = torch.nn.Sequential(   
    torch.nn.Linear(768,800),
    torch.nn.Sigmoid(),
    torch.nn.Linear(800,1),
    torch.nn.Sigmoid()
)

In [ ]:
scoring_function = torch.nn.MSELoss()
num_epochs = 10

optimizer = torch.optim.SGD( model.parameters(), .001 )

In [ ]:
def train():
    for step in range (num_epochs):

        for batch in loader:
            optimizer.zero_grad()
            batchX = batch[0]
            #print(batchX.size())
            batchY = batch[1].unsqueeze(-1)
            #print(batchY.size())
            # do a gradient descent update of out linear model.

            # put data through the model
            preds = model(batchX)
            #print (preds.shape)
            #fixedPreds=preds[:,0]
            #print("preds: ", fixedPreds)
            # collect the error
            errors = scoring_function(preds, batchY)
            #print("errors: ", errors)

            # compute the gradient of the error with regard to the model paramenters
            errors.backward()

            # take a step in that direction
            optimizer.step()
    new_x_tensor= torch.tensor(x_test).float()
    preds= model(new_x_tensor)
    accuracy= accuracy_score(np.round(preds.detach().numpy()), y_test)
    print(accuracy)
        
            

In [ ]:
train()